In [6]:
import pytesseract
import cv2
import numpy as np

In [7]:
images= [i for i in range(1,26)]
print(images)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [8]:
for i in images:
    image = cv2.imread('E:/Coding/CVPARSER/Pictures/'+str(i)+ '.png')
    
    #remove all lines from image
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/gray'+str(i)+ '.png',gray)

    gray = cv2.bitwise_not(gray)
    bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
                                cv2.THRESH_BINARY, 15, -2)

    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/binary'+str(i)+ '.png',bw)

    horizontal = np.copy(bw)
    vertical = np.copy(bw)


    #get the horizontal lines
    cols = horizontal.shape[1]
    horizontal_size = cols // 30
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    horizontal = cv2.erode(horizontal, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure)
    
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/horizontal'+str(i)+ '.png',horizontal)

    #get the vertical lines
    rows = vertical.shape[0]
    verticalsize = rows // 30
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    vertical = cv2.erode(vertical, verticalStructure)
    vertical = cv2.dilate(vertical, verticalStructure)

    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/vertical'+str(i)+ '.png',vertical)


    final = bw - vertical - horizontal
    final = cv2.bitwise_not(final)

    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/final'+str(i)+ '.png',final)


    #get separator locations
    linesImg = vertical + horizontal
    linesImg = cv2.bitwise_not(linesImg)
    edges = cv2.Canny(linesImg, 100, 150)
    minLineLength =2
    maxLineGap = 4
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 150, None, 0, 0)

    for line in lines:
        for x1, y1,x2,y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), (50, 50, 50), 7)


    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/lines'+str(i)+ '.png',linesImg)


    #get multiple blurs
    blursmall = cv2.GaussianBlur(final, (7,5), 1)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/blursmall'+str(i)+ '.png',blursmall)

    threshsmall = cv2.threshold(blursmall, 0,255 , cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/threshsmall'+str(i)+ '.png',threshsmall)

    blurmedium = cv2.GaussianBlur(final, (51,51), 2)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/blurmedium'+str(i)+ '.png',blurmedium)
    threshmedium = cv2.threshold(blurmedium, 0,255 , cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/threshmedium'+str(i)+ '.png',threshmedium)

    blurlarge = cv2.GaussianBlur(final, (7,7), 20)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/blurlarge'+str(i)+ '.png',blurlarge)
    threshlarge = cv2.threshold(blurlarge, 0,255 , cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/threshlarge'+str(i)+ '.png',threshlarge)

    #small
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,3) )
    dilate = cv2.dilate(threshsmall, k , iterations=1)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/dilatesmalls'+str(i)+ '.png',dilate)
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts= cnts[0] if len(cnts)==2 else cnts[1]
    cnts = sorted(cnts, key=lambda x:cv2.boundingRect(x)[0])
    #for j in cnts:
    #    x,y,w,h = cv2.boundingRect(j)
    #    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)

    
    #lines
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (70,1) )
    dilate = cv2.dilate(threshmedium, k , iterations=1)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/dilatemedium'+str(i)+ '1.png',dilate)
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts= cnts[0] if len(cnts)==2 else cnts[1]
    cnts = sorted(cnts, key=lambda x:cv2.boundingRect(x)[0])
    for j in cnts:
        x,y,w,h = cv2.boundingRect(j)
        cv2.rectangle(image,(x,y),(x+w,y+h),(36,255,12),2)

    
    #paragraphs
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (7,4) )
    dilate = cv2.dilate(threshmedium, k , iterations=5)
    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/dilatelarge'+str(i)+ '.png',dilate)
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts= cnts[0] if len(cnts)==2 else cnts[1]
    cnts = sorted(cnts, key=lambda x:cv2.boundingRect(x)[0])

    for j in cnts:
        x,y,w,h = cv2.boundingRect(j)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)

    cv2.imwrite('E:/Coding/CVPARSER/Pictures/Augmented/image'+ str(i)+ '.png',image)

    